In [1]:
from scipy import special
import numpy as np
import matplotlib.pyplot as plt

In [122]:
## object class for node
class node():
    """the class for each node"""
    
    def __init__(self, left, right, data, cnum):
        self.l = left
        self.r = right
        if(left == None and right == None):
            self.n = 1
        else:
            self.n = left.n + right.n
        self.data = data
        self.cluster = cnum

In [3]:
## function to calculate the posterior probability
def p_hyp1(dataset, a):
    # extract the number of features and the total number of data
    if (len(dataset.shape) == 1):
        N = 1
        k = dataset.shape[0]
        print(k)
        # part I
        p1 = 1
        comp = special.gamma(np.sum(dataset)+1) / np.prod(special.gamma(dataset+1))
        p1 = p1 * comp
        
        # part II
        # iterate to calculate the probability
        p2 = p1 * special.gamma(np.sum(a)) / special.gamma(np.sum(dataset) + np.sum(a))
        for j in range(k):
            print(j)
            comp = special.gamma(a[j] + np.sum(dataset[j])) / special.gamma(a[j])
            p2 = p2 * comp
    else:
        N = dataset.shape[0]
        k = dataset.shape[1]
    
        # part I
        p1 = 1
        for i in range(N):
            comp = special.gamma(np.sum(dataset[i, :])+1) / np.prod(special.gamma(dataset[i, :]+1))
            p1 = p1 * comp
        
        # part II
        # iterate to calculate the probability
        p2 = p1 * special.gamma(np.sum(a)) / special.gamma(np.sum(dataset) + np.sum(a))
        for j in range(k):
            comp = special.gamma(a[j] + np.sum(dataset[:, j])) / special.gamma(a[j])
            p2 = p2 * comp

    return p2

In [4]:
## function to calculate the d
def get_d(node, a):
    if node.l == None and node.r == None:
        return a
    else:
        return a*special.gamma(node.n) + get_d(node.l, a)*get_d(node.r, a)

In [5]:
## function to calculate the weight or pi
def get_pi(node, a):
    dk = get_d(node, a)
    pi_k = a*special.gamma(node.n)/dk
    return pi_k

In [6]:
## test dataset
data = np.random.randint(0,4, size=(5,5))
a = [1,1,1,1,1]
data

array([[2, 1, 1, 2, 1],
       [0, 2, 0, 1, 1],
       [1, 0, 0, 3, 2],
       [0, 3, 3, 1, 1],
       [1, 1, 2, 2, 3]])

In [7]:
# get dk
def get_dk(node, a):
    post = p_hyp1(node_new.data, np.repeat(a, data.shape[1]))
    pi = get_pi(node, a)
    if node.l == None and node.r == None:
        return  pi * post
    else:
        return  pi * post + (1-pi) * get_dk(node.l, a) * get_dk(node.r, a)

In [ ]:
with open("wine.csv") as f:
    next(f)
    text = f.read() 

data = []
lines  = text.split('\n')
for line in lines[:-1]:
    arr = line.split(';')
    fl = [int(np.round(float(x))) for x in arr]
    data.append(fl)

data = np.array(data)
data

In [ ]:
data = data[:50,:]

In [159]:
node_list = []
node_list_copy = []
for i in range(data.shape[0]):
    node_list.append(node(None, None, np.array([data[i,:]]), i))
    node_list_copy.append(node(None, None, np.array([data[i,:]]), i))

c = data.shape[0]
a = 1

while c > 1:
    flag = False
    for i in range(len(node_list)):
        for j in range(i+1, len(node_list)):
            #print(node_list[i].data)
            #print(node_list[j].data)
            if len(node_list[i].data.shape) == 1 and len(node_list[j].data.shape) == 1:
                newdata = np.array([node_list[i].data, node_list[i].data])
            else:
                newdata = np.concatenate((node_list[i].data, node_list[j].data), axis = 0)
            #print(newdata)
            node_new = node(node_list[i], node_list[j], newdata, min(node_list[i].cluster,node_list[j].cluster))
            pi_k = get_pi(node_new, a)
            p_hyp = p_hyp1(node_new.data, np.repeat(a, data.shape[1]))
            p_dk = get_dk(node_new, a)
            rk = pi_k * p_hyp / p_dk
            #print(rk)
            if rk > 0.5:
                for k in range(len(node_list_copy)):
                    entry = node_list_copy[k].cluster
                    if entry == node_list[i].cluster or entry == node_list[j].cluster:
                        node_list_copy[k].cluster = min(node_list[i].cluster,node_list[j].cluster)
                node_list = [node_new] + node_list[:i] + node_list[(i+1):j] + node_list[(j+1):]
                #print([node.cluster for node in node_list])
                c = c - 1
                flag = True
                break
        if(flag == True):
            break
    if(flag == False):
        c = 1
    
    

In [160]:
print([node.cluster for node in node_list])

[37, 34, 31, 27, 24, 19, 18, 17, 12, 10, 8, 6, 5, 3, 2, 0, 1, 9, 11, 14, 15, 16, 20, 21, 23, 30, 32, 33, 39, 40, 41, 45, 46, 49]


In [161]:
len(node_list)

34

In [162]:
len(np.unique([node.cluster for node in node_list]))

34

In [163]:
len(np.unique([node.cluster for node in node_list_copy]))

34

In [164]:
[node.cluster for node in node_list_copy]

[0,
 1,
 2,
 3,
 0,
 5,
 6,
 5,
 8,
 9,
 10,
 11,
 12,
 8,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 18,
 23,
 24,
 24,
 2,
 27,
 27,
 6,
 30,
 31,
 32,
 33,
 34,
 31,
 12,
 37,
 17,
 39,
 40,
 41,
 19,
 34,
 3,
 45,
 46,
 37,
 10,
 49]